In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
os.chdir('..')

from src import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [3]:
files = sorted(load_files_from_dataset('01.04.2022_eyedea_RX100'))

In [4]:
output_folder = 'data/manual_annotation_RX100'

In [14]:
for file in files:
    print(file)
    events = load_events(file)
    intervals = load_intervals(file)

    video_file = find_path(f'data/video/**/{file}.*', True)

    rows = [['length', 'n_events', 'interval', 'from_time', 'till_time']]
    for interval_from, interval_till in intervals:        
        n_events = np.sum((events >= interval_from) & (events < interval_till))
        interval_length = interval_till - interval_from
        to_minutes = lambda x: f'{x // 60:02.0f}.{x % 60:02.0f}'

        row = [f'{interval_length:>.1f}', f'{n_events}', 
            f'{to_minutes(interval_from)} : {to_minutes(interval_till)}', 
            f'{interval_from:.0f}', f'{interval_till:.0f}'
        ]
        
        target_name = f'{output_folder}/{file}_{interval_from:.0f}_{interval_till:.0f}.MP4'
        ffmpeg_extract_subclip(video_file, interval_from, interval_till, targetname=target_name)
        rows.append(row)
        
    with open(f'{output_folder}/{file}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(rows)

    table = tabulate(rows, headers='firstrow')
    with open(f'{output_folder}/{file}.txt', 'w') as f:
        f.write(table)
   
    print(table)
    print()
    
    

05_RX100
  length    n_events  interval         from_time    till_time
--------  ----------  -------------  -----------  -----------
    28.4           6  00.00 : 00.28            0           28
    37.7           6  00.28 : 01.06           28           66
    18.3           3  01.06 : 01.24           66           84
    20.4           1  01.24 : 01.45           84          105
    23.5           1  01.45 : 02.08          105          128
   104.2          18  02.08 : 03.53          128          233
    36.6           6  03.53 : 04.29          233          269
    21.2           4  04.29 : 04.50          269          290
    31             3  04.50 : 05.21          290          321
    79.5          10  05.21 : 06.41          321          401
    32.1           7  06.41 : 07.13          401          433
    14.5           2  07.13 : 07.27          433          447
    17.2           1  07.27 : 07.45          447          465
    21.4           2  07.45 : 08.06          465          486

In [15]:
show_video('data/for_manual_annotation/20190819-Kutna Hora-L14-out-MVI_0005_17.56_18.35.MP4')